In [34]:
import numpy as np
import os
from os.path import isfile
import keras
from keras.models import Sequential, Model
from keras.layers import Input, Dense, TimeDistributed, LSTM, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, Flatten, Conv2D, BatchNormalization, Lambda
from keras.layers import Input, Dense, Lambda, Dropout, Activation, LSTM, \
        TimeDistributed, Convolution1D, MaxPooling1D,Conv1D,AveragePooling1D, Flatten,GlobalAveragePooling1D,GlobalMaxPooling1D,concatenate
from keras.layers.advanced_activations import ELU
from keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from keras import backend as K
from keras.utils import np_utils
from keras.optimizers import Adam, RMSprop

from keras import regularizers
import tensorflow as tf

import librosa
import librosa.display
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
npzfile = np.load('testing_spectograms.npz')
print(npzfile.files)
X_train = npzfile['arr_0']
y_train = npzfile['arr_1']

['arr_0', 'arr_1']


In [11]:
npzfile = np.load('data/fma_arrays/validation_spectrograms.npz')
print(npzfile.files)
X_valid = npzfile['arr_0']
y_valid = npzfile['arr_1']


['arr_0', 'arr_1']


In [29]:
GENRES = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal',
        'pop', 'reggae', 'rock']

In [36]:
def train_model():

    n_features = 8
    input_shape = (640, 128)
    model_input = Input(input_shape, name='input')
    layer = model_input
    for i in range(3):

        layer = Convolution1D(
                nb_filter=256,
                filter_length=5,
                name='convolution_' + str(i + 1)
            )(layer)
        layer = Activation('relu')(layer)
        layer = MaxPooling1D(2)(layer)

    layer = Dropout(0.5)(layer)
    layer = LSTM(256, return_sequences=True)(layer)
    layer = Dropout(0.5)(layer)
    layer = TimeDistributed(Dense(len(GENRES)))(layer)
    layer = Activation('softmax', name='output_realtime')(layer)
    time_distributed_merge_layer = Lambda(
            function=lambda x: weightedCustom(x), 
            output_shape=lambda shape: (shape[0],) + shape[2:],
            name='output_merged_weighted'
        )
    model_output = time_distributed_merge_layer(layer)
    model = Model(model_input, model_output)
    opt = RMSprop(lr=0.00001)
    model.compile(
            loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy']
        )

    
    model.fit(X_train, y_train, batch_size=32, nb_epoch=100,
              validation_data=(X_valid, y_valid), verbose=1)

    return model

In [31]:
def weightedCustom(x):
    weights=np.arange(77)
    weights=weights[:,np.newaxis]
    weightsKeras=K.variable(value=weights)
    values_tensor=x
    out=values_tensor*weights
    out=K.sum(out,axis=1)
    out=tf.divide(out,3003)#1+2+3+.....+77
    return out

In [37]:
train_model()

C:\Users\Nicsi\Anaconda3\envs\ML_env\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(name="convolution_1", filters=256, kernel_size=5)`
  if sys.path[0] == '':
C:\Users\Nicsi\Anaconda3\envs\ML_env\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(name="convolution_2", filters=256, kernel_size=5)`
  if sys.path[0] == '':
C:\Users\Nicsi\Anaconda3\envs\ML_env\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(name="convolution_3", filters=256, kernel_size=5)`
  if sys.path[0] == '':


ValueError: Dimensions must be equal, but are 76 and 77 for 'output_merged_weighted_5/mul' (op: 'Mul') with input shapes: [?,76,10], [77,1].